<a href="https://colab.research.google.com/github/protocol-streams/querent-experimental/blob/main/llama2_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://swharden.com/blog/2023-07-29-ai-chat-locally-with-python/

In [1]:
!pip install llama-cpp-python==0.1.78

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.4 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.78-cp310-cp310-linux_x86_64.whl size=296587 sha256=b611ec93e85005f812161bb9df6247ebf86b2e11222da723931399bbbf0ab792
  Stored in directory: /root/.cache/pip/wheels/61/f9/20/9ca660a9d3f2a47e44217059409478865948b5c8a1cba70030
Successfully built llama-cpp-python


In [2]:
# Connect to Google Colab
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#%cd drive/MyDrive/corpus-data/

In [3]:
from llama_cpp import Llama

from IPython.display import display, HTML
import json
import time
import pathlib

Load two different models so we can compare their responses to the same prompt.

Note that `n_ctx` is the maximum number of context tokens, and increasing this value increases the maximum length of the responses.

In [4]:
%cd drive/MyDrive/LLAMA2/

/content/drive/MyDrive/LLAMA2


In [5]:
%pwd

'/content/drive/MyDrive/LLAMA2'

In [6]:
MODEL_Q8_0 = Llama(
    model_path="llama-2-7b-chat.ggmlv3.q8_0.bin",
    n_ctx=2048)

MODEL_Q2_K = Llama(
    model_path="llama-2-7b-chat.ggmlv3.q2_K.bin",
    n_ctx=2048)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 
AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


In [9]:
def query(model, question):
    model_name = pathlib.Path(model.model_path).name
    time_start = time.time()
    #prompt = f"Q: {question} A:"
    prompt = f"""[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always say yes Sir before you answer and answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

{question} [/INST]"""
    output = model(prompt=prompt, max_tokens=0) # if max tokens is zero, depends on n_ctx
    response = output["choices"][0]["text"]
    time_elapsed = time.time() - time_start
    display(HTML(f'<code>{model_name} response time: {time_elapsed:.02f} sec</code>'))
    display(HTML(f'<strong>Question:</strong> {question}'))
    display(HTML(f'<strong>Answer:</strong> {response}'))
    print(json.dumps(output, indent=2))

In [10]:
query(MODEL_Q2_K, """Given the text - (Glacier mass variations have a direct impact on some of the key components of the
global water cycle, including sea level rise and freshwater availability. Apart from being
one of the largest Himalayan glaciers, Gangotri is one of the sources of water for the
Ganges river, which has a considerable influence on the socioeconomic structure of a
largely over-populated catchment area accounting for ∼26% of India’s landmass)- can you identify some entities and their relationships?""")

Llama.generate: prefix-match hit


{
  "id": "cmpl-b66a756a-a3eb-4fcd-9185-9ee6604dae47",
  "object": "text_completion",
  "created": 1693504431,
  "model": "llama-2-7b-chat.ggmlv3.q2_K.bin",
  "choices": [
    {
      "text": "  Yes, Sir. Based on the text provided, there are several entities and their relationships that can be identified:\n\n1. Glaciers - Glaciers are the primary entity mentioned in the text, and they have a direct impact on various components of the global water cycle, including sea level rise and freshwater availability.\n2. Himalayan glaciers - Gangotri is specifically identified as one of the largest Himalayan glaciers, indicating that these glaciers play a significant role in the global water cycle.\n3. Water sources - Gangotri is also mentioned as a source of water for the Ganges river, which has a considerable influence on the socioeconomic structure of the largely over-populated catchment area accounting for \u223c26% of India\u2019s landmass. This suggests that the water from Gangotri and oth

In [ ]:
query(MODEL_Q8_0, """Given the text - (Glacier mass variations have a direct impact on some of the key components of the
global water cycle, including sea level rise and freshwater availability. Apart from being
one of the largest Himalayan glaciers, Gangotri is one of the sources of water for the
Ganges river, which has a considerable influence on the socioeconomic structure of a
largely over-populated catchment area accounting for ∼26% of India’s landmass. In this
study, we present the most recent assessment of the Gangotri glacier dynamics, combin-
ing the use of interferometric techniques on synthetic aperture radar data and sub-pixel
offset tracking on Advanced Spaceborne Thermal Emission and Reflection Radiometer
(ASTER) satellite imagery. Results show that on average, the Gangotri glacier snout
has receded at a rate of 21.3 ± 3 m year−1 over a period of 6 years (2004–2010).
While glacier surface velocity near the snout is estimated to be between 24.8 ± 2.3 and
28.9 ± 2.3 m year−1 , interior portions of the glacier recorded velocities in the range of
13.9 ± 2.3 to 70.2 ± 2.3 m year−1 . Further, the average glacier surface velocity in the
northern (lower) portions (28.1 ± 2.3 m year−1 ) is observed to be significantly lower
than in the southern (higher) portions (48.1 ± 2.3 m year−1 ) of the Gangotri glacier.
These values are calculated with an uncertainty of less than 5 m year−1 . Results also
highlight a consistent retreat and non-uniform dynamics of the Gangotri glacier me money)- can you identify some entities and their relationships?""")

{
  "id": "cmpl-5c5cb407-854d-4ea3-b3f7-2cefdcdf365c",
  "object": "text_completion",
  "created": 1693332574,
  "model": "llama-2-7b-chat.ggmlv3.q8_0.bin",
  "choices": [
    {
      "text": " Certainly! Here are some entities and their relationships that can be identified from the given text:\n\nEntities:\n\n1. Glacier mass variations - have a direct impact on some key components of the global water cycle, including sea level rise and freshwater availability.\n2. Gangotri glacier - one of the sources of water for the Ganges river, which has a considerable influence on the socioeconomic structure of a largely over-populated catchment area accounting for \u223c26% of India\u2019s landmass.\n3. Interferometric techniques - used to study Gangotri glacier dynamics combining the use of satellite data.\n4. Sub-pixel offset tracking - used on Advanced Spaceborne Thermal Emission and Reflection Radiometer (ASTER) satellite imagery to study Gangotri glacier dynamics.\n5. Glacier surface veloci